# OGC API Processes demonstratie client

De Pygeoapi server kun je starten met `docker compose up`

Deze zou dan beschikbaar moeten zijn op http://localhost:5000/


In [1]:
import folium
import requests  # For making HTTP requests
import geopandas as gpd
import json

In [ ]:
# processes landing page (json)

url = "http://localhost:5000/processes/"  
 
response = requests.get(url)

response.json()

{'processes': [{'version': '0.1.0',
   'id': 'squared',
   'title': 'Squared processor',
   'description': 'An example process that takes a number or integer and returns the squared result',
   'jobControlOptions': ['sync-execute', 'async-execute'],
   'keywords': ['squared'],
   'links': [{'type': 'text/html',
     'rel': 'about',
     'title': 'information',
     'href': 'https://example.org/process',
     'hreflang': 'en-US'},
    {'type': 'application/json',
     'rel': 'self',
     'href': 'http://localhost:5000/processes/squared?f=json',
     'title': 'Process description as JSON',
     'hreflang': 'en-US'},
    {'type': 'text/html',
     'rel': 'alternate',
     'href': 'http://localhost:5000/processes/squared?f=html',
     'title': 'Process description as HTML',
     'hreflang': 'en-US'},
    {'type': 'text/html',
     'rel': 'http://www.opengis.net/def/rel/ogc/1.0/job-list',
     'href': 'http://localhost:5000/jobs?f=html',
     'title': 'Jobs list as HTML',
     'hreflang': '

In [27]:
def handle_response(response):
  if response.status_code == 200:
    
    return response.json()
    
  else:
    print(f"Error: {response.status_code, response.json()['description']}")

In [ ]:
# alleen een lijstje met titels en id's van de processen

url = "http://localhost:5000/processes/"  

headers = {"Content-Type": "application/json"}

response = requests.get(url, headers=headers)
processes =  handle_response(response)

for title in processes['processes']:
    print(title['title'], ' - ID:  ', title['id'])

Squared processor  - ID:   squared
Local outlier factor (LOF)  - ID:   localoutlier
Shapely Functions  - ID:   shapely-functions
Geometry Details  - ID:   geom-details


In [ ]:
# details van een specifiek proces

url = "http://localhost:5000/processes/geom-details"

headers = {"Content-Type": "application/json"}

response = requests.get(url, headers=headers)
response.json()

{'version': '0.2.0',
 'id': 'geom-details',
 'name': 'geom-details',
 'title': 'Geometry Details',
 'description': 'An example process that takes a WKT Polygon geometry as input and return information such as Area, Bounds, center',
 'keywords': ['polygon', 'geometry'],
 'links': [{'type': 'text/html',
   'rel': 'about',
   'title': 'information',
   'href': 'https://shapely.readthedocs.io/en/stable/reference/shapely.Polygon.html',
   'hreflang': 'en-US'},
  {'type': 'application/json',
   'rel': 'self',
   'href': 'http://localhost:5000/processes/geom-details?f=json',
   'title': 'Process description as JSON',
   'hreflang': 'en-US'},
  {'type': 'text/html',
   'rel': 'alternate',
   'href': 'http://localhost:5000/processes/geom-details?f=html',
   'title': 'Process description as HTML',
   'hreflang': 'en-US'},
  {'type': 'text/html',
   'rel': 'http://www.opengis.net/def/rel/ogc/1.0/job-list',
   'href': 'http://localhost:5000/jobs?f=html',
   'title': 'Jobs list as HTML',
   'hrefla

In [ ]:
# uitvoeren van een proces

url = "http://localhost:5000/processes/geom-details/execution"  
data = {"inputs": {"polyWKT": "POLYGON ((73.65865031385647 17.77623164372008, 73.65897114729012 17.777001795974, 73.65825595609473 "
"17.776963606683765, 73.65817351972652 17.776647483916918, 73.65852108927962 17.7764247125032, 73.65865031385647 17.77623164372008))"}}

headers = {"Content-Type": "application/json"}

response = requests.post(url, json=data, headers=headers)

response.json()

{'result': {'area': 3.426211280810428e-07,
  'bounds': [73.65817351972652,
   17.77623164372008,
   73.65897114729012,
   17.777001795974],
  'centroid': 'POINT (73.65856394829866 17.776717111088377)'}}

## Kaart met de waarneemstations (OGC API Features)

In [26]:
m = folium.Map(location=[52.2, 5.5], zoom_start=8)

url = "http://localhost:5000/collections/knmi_meetstations/items?f=json&limit=1000"

# Fetch the GeoJSON data
gdf = gpd.read_file(url)

#quick hack omdat de datetime kolom niet goed naar geojson geconverteerd wordt...
gdf_subset = gdf[['STN','TYPE', 'geometry']]

# Convert to GeoJSON
geojson = gdf_subset.to_json()

folium.GeoJson(
    gdf_subset,
    name="Stations",
    style_function=lambda x: {'fillColor': 'blue', 'color': 'black', 'weight': 1, 'fillOpacity': 0.5},
    tooltip=folium.GeoJsonTooltip(fields=["STN"]),
    popup=folium.GeoJsonPopup(fields=["STN", "TYPE"]),
    highlight_function=lambda x: {"fillOpacity": 0.8},
).add_to(m)

m

Kaart met het resultaat van de classificatie door het OGC API Processes localoutlier algoritme 

In [28]:
def calc_lof(feat):
    url = "http://localhost:5000/processes/localoutlier/execution"  
    data = {"inputs": {"dataset": feat}}
    headers = {"Content-Type": "application/json"}

    response = requests.post(url, json=data, headers=headers)
    return handle_response(response)


#let op: omdat dit de url is vanaf de ogc api process functie naar de ogc api features op dezelfde server is de url zonder poort 5000
input = 'http://localhost/collections/knmi_meetstations/items?f=json&limit=1000' 
result = calc_lof(input)

print(result)
with open('meetstations.geojson', 'w') as f:
    json.dump(json.loads(result['value']), f)

m = folium.Map(location=[52.2, 5.5], zoom_start=8)
gdf = gpd.read_file('meetstations.geojson')
gdf2 = gdf[['STN','TYPE', 'geometry','abnormality']]


folium.GeoJson(
    gdf2,
    name="Stations",
    marker=folium.Marker(icon=folium.Icon(icon='star')),
    tooltip=folium.GeoJsonTooltip(fields=["STN","abnormality"]),
    popup=folium.GeoJsonPopup(fields=["STN", "TYPE"]),
    style_function=lambda x: {
        'markerColor': 'orange' if x['properties']['abnormality'] == -1 else 'green',
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.7
    },
    highlight_function=lambda x: {"fillOpacity": 0.8},
).add_to(m)

value_counts = gdf['abnormality'].value_counts()
print(value_counts)
m

{'id': 'output_dataset', 'value': '{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"STN": 201, "TYPE": "Platform/AWS", "abnormality": -1}, "geometry": {"type": "Point", "coordinates": [2.9358, 54.3256]}}, {"id": "1", "type": "Feature", "properties": {"STN": 203, "TYPE": "Platform/AWS", "abnormality": 1}, "geometry": {"type": "Point", "coordinates": [3.3417, 52.36]}}, {"id": "2", "type": "Feature", "properties": {"STN": 204, "TYPE": "Platform/AWS", "abnormality": 1}, "geometry": {"type": "Point", "coordinates": [3.6278, 53.2694]}}, {"id": "3", "type": "Feature", "properties": {"STN": 205, "TYPE": "Platform/AWS", "abnormality": -1}, "geometry": {"type": "Point", "coordinates": [3.8103, 55.3992]}}, {"id": "4", "type": "Feature", "properties": {"STN": 206, "TYPE": "Platform/AWS", "abnormality": -1}, "geometry": {"type": "Point", "coordinates": [4.0122, 54.1167]}}, {"id": "5", "type": "Feature", "properties": {"STN": 207, "TYPE": "Platform/AWS", "abn